# Clustering aerator import data with k-means

In [164]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    silhouette_score,
    adjusted_rand_score,
    calinski_harabasz_score,
)
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
import warnings

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")


In [165]:
df_2021 = pd.read_csv(
    "../../../data/raw/aquaculture/aerator_imports/2021_aerator_imports.csv"
)
df_2022 = pd.read_csv(
    "../../../data/raw/aquaculture/aerator_imports/2022_aerator_imports.csv"
)
df_2023 = pd.read_csv(
    "../../../data/raw/aquaculture/aerator_imports/2023_aerator_imports.csv"
)
df_2024 = pd.read_csv(
    "../../../data/raw/aquaculture/aerator_imports/2024_aerator_imports.csv"
)
df = pd.concat([df_2021, df_2022, df_2023, df_2024], ignore_index=True)

# Preview datatype and shape
print(df.dtypes)
print(df.shape)

# Check for missing values, exclude 0 missing values columns
df = df.loc[:, df.isnull().mean() < 0.5]
print("Missing values in each column:")
print(df.isnull().sum())

# Display the most common values in the 'DESCRIPCION PRODUCTO COMERCIAL' column
print("Most common values in 'DESCRIPCION PRODUCTO COMERCIAL':")
print(df["DESCRIPCION PRODUCTO COMERCIAL"].value_counts().head(10))


DIA                          int64
MES                          int64
AÑO                          int64
REGIMEN ADUANERO            object
REFRENDO                    object
                            ...   
DESCRIPCIóN ARANCELARIA     object
PAíS DE EMBARQUE            object
FECHA SALIDA                object
TOTAL AJUSTES              float64
TOTAL OTROS AJUSTES        float64
Length: 64, dtype: object
(4511, 64)
Missing values in each column:
DIA                                  0
MES                                  0
AÑO                                  0
REGIMEN ADUANERO                     0
REFRENDO                             0
DAU                                385
ITEM                                 0
RUC IMPORTADOR                       0
PROBABLE IMPORTADOR                  0
PAIS DE PROCEDENCIA                  0
CIUDAD EMBARQUE                    158
VIA DE TRANSPORTE                    0
ADUANA                               0
PARTIDA ARANCELARIA                  0
D

In [166]:
# Dictionary to map Aerator names that are treated as distinct when they are actually the same:
description_mapping = {
    "AERADORES": "AIREADOR",
    "AIREADORES": "AIREADOR",
    "AEREADORES COMPLETO": "AIREADOR",
    "AEREADOR ELECTRICO COMPLETO": "AIREADOR",
    "AIREADOR DE RUEDA DE PALETA": "AIREADOR",
    "AEREADORES": "AIREADOR",
    "AIREADOR DE 16 PALETAS": "AIREADOR",
    "AERATOR 16 PADDLE": "AIREADOR",
    "AEREADOR": "AIREADOR",
    "AIREADOR DE PALETAS": "AIREADOR",
    "PADDLEWHEEL AERATOR": "AIREADOR",
    "AIREADOR DE 8 PALETAS": "AIREADOR",
    "MAQUINA AIERADOR AQUAMIX B-603": "AIREADOR",
    "AIREADORES 3.5 HP 440V": "AIREADOR",
    "AIREADORES DE 16 PALETAS PARA MEZCLAR OXIGENO CON MOTOR": "AIREADOR",
    "AERATOR 8 PADDLE": "AIREADOR",
    "AIREADOR DE 4 PALETAS": "AIREADOR",
    "Aireador electrico": "AIREADOR",
    "AIREADORES DE 12 PALETAS PARA MEZCLAR OXIGENO CON MOTOR": "AIREADOR",
    "AIREADOR DE 12 PALETAS": "AIREADOR",
    "AERATOR": "AIREADOR",
    "AEREADOR 4 PADDLE": "AIREADOR",
    "AIREADORES AUTOMATICOS": "AIREADOR",
    "AIREADORES ELECTRICOS": "AIREADOR",
    "AIREADOR PAD 3 HP SS MOTOR 8 IMP 440V": "AIREADOR",
    "AIREADORES 2 HP 460V": "AIREADOR",
    "SET DE AIREADORES": "AIREADOR",
    "AIREADOR DE PALETA": "AIREADOR",
    "AIREADOR 16 PADDLE": "AIREADOR",
    "AEREADOR 16 PADDLE": "AIREADOR",
    "DIESEL PADDLEWHEEL AERATORS": "AIREADOR",
    "AIREADORES PARA PISCINA CAMARONERA": "AIREADOR",
    "AERATOR 4 PADDLE": "AIREADOR",
    "AIREADOR AQUAMIX": "AIREADOR",
    "AIREADOR DIESEL 16HP": "AIREADOR",
    "PADDLE WHEEL AERATOR/ AIREADOR": "AIREADOR",
    "AEREADOR 8 PADDLE": "AIREADOR",
    "AIR JET AERATOR": "AIREADOR",
    "AIREADOR 3.5HP 8 PALETAS 440V": "AIREADOR",
    "AIREADOR 3.5HP 440V 8 PALETAS CUERPO REDUCTOR PLASTICO CON COJINETE": "AIREADOR",
    "AERADORES PARA PISCINAS DE CAMARON(SECTOR ACUICOLA) CON SUS PARTES Y ACCESORIOS": "AIREADOR",
    "AIREADOR SOLAR PARA USO ACUICOLA": "AIREADOR",
    "8 IMPELLER AERATOR MOTOR FLOAT": "AIREADOR",
    "AIREADOR SS MOTOR 6 IMP": "AIREADOR",
    "AIREADORES 3 HP 460V": "AIREADOR",
    "AIREADOR SUMERGIBLE": "AIREADOR",
    "AIREADORES DE 42 PALETAS PARA USO EN PISCINAS ACUICOLAS": "AIREADOR",
    "PADDLE AERATORS, 2.6 KW MOTOR 60 HZ": "AIREADOR",
    "AERADORES 3HP 6 PALETAS": "AIREADOR",
    "8 IMPELLER AERATOR": "AIREADOR",
    "AIREADOR PALETAS 18HP 16 PALETAS MOTOR DIESEL": "AIREADOR",
    "AIREADOR PALETAS 4HP 12 PALETAS 440V": "AIREADOR",
    "AIREADOR PALETAS 3HP 8 PALETAS 440V": "AIREADOR",
    "AIREADOR PALETAS 3HP 6 PALETAS 440V": "AIREADOR",
    "AIREADOR PALETAS 2HP 4 PALETAS 440V": "AIREADOR",
    "AIREADOR PALETAS 2HP 4 PALETAS 220V": "AIREADOR",
    "WHEELS PADDLE WHEEL AERATOR": "AIREADOR",
    "AIREADOR TRIFASICO 3HP 440V 60HZ": "AIREADOR",
    "AIREADOR SPLASH 1.5HP 220V": "AIREADOR",
    "AIREADOR ELECTRICO 5HP 16 PALETAS": "AIREADOR",
    "AIREADOR ELECTRICO 4HP 12 PALETAS": "AIREADOR",
    "AIREADOR ELECTRICO 3HP 8 PALETAS": "AIREADOR",
    "AIREADOR ELECTRICO 3HP 6 PALETAS": "AIREADOR",
    "AIREADOR ELECTRICO 2HP 4 PALETAS": "AIREADOR",
    "AIREADOR ELECTRICO PARA MEZCLAR OXÍGENO 3HP": "AIREADOR",
    "AIREADOR ELECTRICO PARA MEZCLAR OXÍGENO 2HP": "AIREADOR",
    "AIREADOR ELECTRICO PARA MEZCLAR OXÍGENO 1HP": "AIREADOR",
    "AIREADORES DE 16 PALETAS PARA MEZCLAR OXIGENO ELÉCTRICO": "AIREADOR",
    "AIREADORES DE 16 PALETAS PARA MEZCLAR OXIGENO SIN MOTOR": "AIREADOR",
    "AIREADOR ELECTRICO PARA MEZCLAR OXÍGENO 4HP": "AIREADOR",
    "AIREADOR PAD 5 HP SS MOTOR 16 IMP 440v 60": "AIREADOR",
    "16 WHEELS WITHOUT GEARBOX AND DIESEL ENGINE": "AIREADOR",
    "EIGHT PADDLE WHEEL AEREATOR": "AIREADOR",
    "AIREADOR DE ENERGIA": "AIREADOR",
    "AIREADOR SOLAR": "AIREADOR",
    "AEREADOR SUMERGIBLE": "AIREADOR",
    "AERATOR, 16 IMPELLER AERATOR": "AIREADOR",
    "AIREADOR AQUA-UNIT FLOAT DE 3HP TRIFÁSICO 230/460V 60HZ 1.5 SF": "AIREADOR",
    "MULTI-IMPULSOR AIREADOR DE PALETAS PA-308": "AIREADOR",
    "AIREADOR DE RUEDA DE PALAS": "AIREADOR",
    "AIREADORES DE 16 PALETAS PARA MEZCLAR OXIGENO": "AIREADOR",
    "4HP AIREADORES PARA MEZCLAR OXIGENO": "AIREADOR",
    "AQUA UNI-FLT 3 HP 230/460 V 60 HZ 3PH 1.15 SF": "AIREADOR",
    "AERATOR Y PADDLE": "AIREADOR",
    "PADDLEWHEEL AERATOR 8 IMPELLER": "AIREADOR",
    "IMPELLER REGULAR 8 IMPELLER": "AIREADOR",
    "4.5HP IMPELLER AERATOR S.S.SC-3.0": "AIREADOR",
}

# Values in the 'DESCRIPCION PRODUCTO COMERCIAL' we can filter out
filter_out = [
    "AGITADOR",
    "AGITADOR MAGNETICO",
    "SISTEMA DE FABRICACIoN DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACION DE ALIMENTO BALANCEADO PARA CAMARoN A TRAVeS DE LA MOLIENDA (DESMONTADO)",
    "MOLEDORA METALICA",
    "MEZCLADORA",
    "MEZCLADOR",
    "MAQUINA MEZCLADORA",
    "BATIDORA",
    "AGITADOR PARA LABORATORIO",
    "MAQUINA TRITURADORA",
    "AGITADOR DIGITAL PARA LABORATORIO",
    "MEZCLADOR VORTEX",
    "MAQ AGITADORA MEZCLADORA",
    "SISTEMA DE FABRICACIoN DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACIoN DE ALIMENTO BALANCEADO PARA CAMARoN SIN MONTAR",
    "LICUADORA INDUSTRIAL",
    "MOLINO",
    "AGITADOR CALENTADOR",
    "MEZCLADOR PARA LABORATORIO",
    "MEZCLADOR ESTATICO",
    "APARATO AGITADOR CON PLACA CALEFACTORA",
    "MEZCLADOR DE SANGRE",
    "HOMOGENIZADOR",
    "MOTOVIB",
    "AGITADOR ORBITAL",
    "MOLEDORA ACRILICA",
    "EQUIPO PARA MEZCLAS COLORANTES",
    "MEZCLADOR DE PINTURA",
    "MAQUINA MEZCLADORA DE PINTURA",
    "MESCLADORAS",
    "MOLINO DE BOLAS",
    "VORTEX",
    "MIXING MACHINE",
    "AMASADORA",
    "RASPADORA DE HIELO",
    "MIXER",
    "MEZCLADOR 1400W 800RP MINGCO",
    "VORTEX 0 2500 RPM 115V",
    "TOPLVL XPRO SOLV UL",
    "AGITADOR VORTEX",
    "MEZCLADOR PARA TALADRO",
    "MOLINILLO DE CAFÉ",
    "TRITURADORA",
    "HOMOGENIZADOR DE TUBOS",
    "AGITADOR DE VARILLA PARA MUESTRAS",
    "APARATO EMULSIONADOR DE GRASAS",
    "VORTEX DIGITAL 500 RPM 3000 RPM 30392122",
    "TRITURADOR",
    "Agitador Mezclador",
    "MINI AGITADOR MAGNETICO",
    "EMULSIONADOR DE SALSAS",
    "BATIDORA DE USO INDUSTRIAL HSIAO",
    "16WHEELS WITHOUT GEARBOX AND DIESEL ENGINE WITHOUT REDUCER SHAFT CUT INTO 2PCS RE CONNECTED BY SHAFT SS304 PIPE CONNECTOR WITH FLAT RING ON THE PADS",
    "MAGNETIC STIRRER",
    "MAQUINA PARA TINTAS CE-MN10-85",
    "AGITADORES",
    "AGITADOR NEUMATICO",
    "MEZCLADOR DE RESINA",
    "AGITADOR DE LABORATORIO",
    "VORTEX MIXER",
    "MAQUINAS AGITADORAS MAGNETICA",
    "PACK COMPLETO HOMOGENEIZADOR",
    "TAMIZ DE MAQUINA",
    "AGITADOR INDUSTRIAL PORTATIL",
    "ACCESORIOS AGITADOR MAGNETICO",
    "APARATO DE MEZCLA",
    "PROCESADOR DE ALIMENTOS",
    "PLATO AGITADOR",
    "MOLINO DE CAFE",
    "PAINT MIXING MACHINE",
    "MEZCLADOR DE TINTA",
    "AGITADOR VRP0509S080",
    "TAMIZADORA CENTRIFUGA",
    "PICATODO",
    "BATIDORA INDUSTRIAL",
    "MAQUINA DOSIFICADORA",
    "16 WHEELS WITHOUT GEARBOX AND DIESEL ENGINE ( WITHOUT REDUCER SHAFT CUT INTO 2 PCS RE CONNECTED BY SHAFT DOUBBLE RUBBER CONNECTOR WITH FLAT RING ON THE PADS FUDU MOVABLE JOINT 2M TRANSMISSION SHAFT 4MM 7 PCS 175 FLOATER 68CM IMPELLER NSK MOVABLE JOINT",
    "ACCESORIO PARA MOLINO",
    "MICROPIPETA VOLUMEN VARIABLE PARA LABORATORIO",
    "REVOLVEDOR PARA PINTURA 4X59",
    "AGITADOR MAGNETICO CON CALEFACCION",
    "CONCENTRADOS",
    "AGITADOR VORTEX TIPO MX SUSA PARA USO EN LABORATORIO DE 110 V 60HZ",
    "TAMIZADORA",
    "AGITADOR DE TUBOS",
    "MEZCLADORES",
    "PUNTAS PARA MEZCLAR SILICONA",
    "TRITURADORA DE HIELO",
    "MAQUINA AGITADORA",
    "MEZCLADORA CONCENTRADOS",
    "Mezclador Vortex",
    "MOLINO COLOIDAL",
    "MEZCLADOR CON VALVULAS Y ACCESORIOS",
    "BEADS MIXER MECHANISM UNIT",
    "SOPORTE PARA AGITADOR DE PLACA",
    "SOPORTE PARA AGITADOR DE PLACA",
    "MORTAR GRIBER RM200",
    "MORTAR GRINDER RM 200",
    "AGITADOR MAGNETICO AREC",
    "AGITADOR MAGNETICO HSC",
    "Static mixer",
    "MILL ZM200",
    "SIEVE SHAKER AS 200 CONTROL",
    "SIEVE SHAKER AS 200 CONTROL",
    "AGITADOR DE PLACA CALIENTE",
    "PLATO CALENTADOR-AGITADOR",
    "SOPORTE PARA AGITADOR DE PLACA",
    "BOECO SUNFLOWER",
    "BOECO VORTEX",
    "PLANTA DE TRATAMIENTO RAEE DE OCASION",
    "HOMOGENIZADOR PARA LABORATORIO",
    "MEZCLADOR GIROSCPICO",
    "MEZCLADOR GIROSCPICO",
    "MEZCLADOR GIROSCPICO",
    "MEZCLADOR LABORATORIO",
    "Mezclador a inyeccion",
    "CALENTADOR AGITADOR PLATO DE PORCELANA",
    "AGITADORES ELECTRICOS",
    "TURBOMEZCLADOR GRANULADOR",
    "MINI MEZCLADOR PARA LABORATORIO",
    "BATIDORA DE HARINA",
    "MOLINO DE CARNE",
    "MOLINO DE CARNE",
    "amasadora",
    "MAQUINA DE TAMIZ DE POLVO",
    "MAQUINA DE GRANULADO PELLET",
    "SLB-80 100235656",
    "Wall ring type 100235657",
    "AGITADOR MEZCLADOR DE LABORATORIO",
    "KIT DE MOLEDORES",
    "HERRAMIENTA",
    "LICUADORA PARA LABORATORIO",
    "OSCILADOR",
    "EQUIPO AGITADOR DE MUESTRAS DE LABORATORIO",
    "CILINDRO ACERO",
    "AGITADOR DE CALOR",
    "AGITADOR MAGNETICO",
    "MEZCLADORA DE ESTIMULACION",
    "PLASTIC MIXING MACHINE",
    "ELECTRIC MIXER",
    "TRITURADORA DE DOBLE EJE",
    "AGITADORES MAGNETICOS MAQUINITA",
    "MIXER MAQUINA PARA MEZCLAR MAIZ",
    "CRUSHER MAQUINA PARA TRITURAR MAIZ",
    "MAQUINA MEZCLADORA DE ARTEMIA",
    "MOLINO DE CUCHILLAS",
    "AGITADOR MAGNTICO",
    "VIBRADOR POKER",
    "CORTADOR DE CONCRETO",
    "MEZCLADORA DE PALETAS MZPT 6300 2X40HP",
    "LABORATORY ROLL MIXER INCLUDING ACCESSORIES",
    "SET SECAD0R DE MATERIAL CON TOLVA SHD-100EB",
    "removedora industrial",
    "MOLINOS",
    "ACCESORIO",
    "ACCESORIO PARA REBANADORA",
    "PLATO AGITADOR CALENTADOR 10X10",
    "AGITADOR SUBMERSIBLE",
    "Licuadora industrial CB-968",
    "PLANCHA DE AGITACION CON CALENTAMIENTO",
    "LIMPIADOR POR ULTRASONIDO",
    "AGITATOR ALTB SANITARY",
    "MEZCLADOR PARA TALADRO TIPO ALETA 6 CM",
    "Maquina granizadora SM-102",
    "DOSIFICADOR MANUAL Y AGITADOR",
    "MAQUINAS AGITADORAS DE POLVO",
    "TAMIZ INDUSTRIAL",
    "PLANCHA DE AGITACION MAGNETICA",
    "AGITADOR VORTEX TIPO MX SUSA PARA USO EN LABORATORIO DE 110 V 60 HZ",
    'REVOLVEDOR PARA PINTURA 4X59"',
    "MEZCLADOR ESPIRAL",
    "PLATO CALIENTE CON AGITACION PARA LABORATORIO",
    "MEZC. ELECT. HM 140 - NEMA (CON MANDRIL) 127 V",
    "MEZCLADOR DE ALIMENTOS",
    "Licuadora de fluido",
    "KIT COJINETE",
    "VARRILLA AGITADORA",
    "PLANCHA DE AGITACION",
    "MAQUINA GRANIZADORA",
    "REVOLVEDOR PARA PINTURA 2X15",
    "PACK COMPLETO ULTRATURRAX",
    "TRITURADOR DE HIELO ELECTRICA",
    "AGITADOR MAGNETICO CON PLACA CALEFACTORA",
    "BATIDORA DE HELADO SOFT",
    "RASPADORA DE HIELO DE 4 CUCHILLAS",
    "3 HP MOTOR",
    "MOTOR AGITADOR",
    "PLAN DE TAMIZ",
    "MEZCLADORA DE RODILLO PARA LIQUIDOS CON ACCESORIOS",
    "MAQUINA PARA TRITURAR HIELO",
    "MOLINO DE PLASTICO",
    "APARATO AGITADOR MAGNETICO",
    "REDUCTOR PLASTICO",
    "MOTOR",
    "HOMOGENIZADOR DIGITAL PARA LABORATORIO",
    "AGITADOR AREC",
    "MEZCLADOR DE MORTERO HM 120",
    "REVOLVEDOR PARA PINTURA 2X15",
    "MEZCLADOR PARA TALADRO TIPO COPA 10 CM",
    "DOSIFICADOR DE MATERIAL",
    "TANQUE MEZCLADOR",
    "AMASADORA DE 50 LIBRAS",
    "AMASADORA DE 30 LIBRAS",
    "EMBARQUE PARCIAL DE SISTEMA DE FABRICACION DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACION DE ALIMENTO BALANCEADO PARA CAMARON",
    "DILUTOR DE MEZCLA",
    "HOMOGENEIZADOR DE LABORATORIO",
    "MEZCLADOR ELECTRICO",
    "CUTTER",
    "VORTEX DIGITAL",
    "SMALL MIXING LID",
    "REVOLVEDOR PARA PINTURA 2X15",
    "MOLINO PARA PLASTICO",
    "TRITURADOR DE HIELO 110V/60HZ",
    "HOMOGENEIZADOR",
    "SISTEMA DE FABRICACIoN DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACION DE ALIMENTO BALANCEADO PARA CAMARoN A TRAVeS DE LA MOLIENDA (DESMONTADO)SISTEMA DE FABRICACIoN DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACION DE ALIMENTO BAL",
    "AGITADOR DE PLACAS",
    "TAMIZ DE LABORATORIO",
    "AGITADOR VORTEX MIXER ZX3",
    "ROTADOR DE TUBOS",
    "SISTEMA DE FABRICACION DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACION DE ALIMENTO BALANCEADO PARA CAMARON SIN MONTAR",
    "MANTA BIOBASE HME I 500ML ANALOGICA DE CALENTAMIENTO",
    "MOLINOS PARA TRITURACIÓN DE ARTÍCULOS PLÁSTICOS",
    "ACCESORIO MOLINO",
    "ETP - MIXER,VARIMIXER 60 QUART ,MARK IV 208/60/3 NO HUB AND ACCESORIES",
    "AGITADOR AUTOMATICO Y DISPENSADOR MANUAL",
    "UPPER ANVIL 8768",
    "LICUADORA",
    "PLATO CALENTADOR AGITADOR MS7 H550 VOLTAJE 110 V 60 HZ",
    "TAPA DE MEZCLADORA",
    "BATIDORA DE 10 LITROS",
    "BATIDORA DE 15 LITROS",
    "BATIDORA DE 20 LITROS",
    "BATIDORA DE 30 LITROS",
    "MEZCLADOR DE 20 LIBRAS",
    "MOTOR 2HP 440V",
    "BATIDORA DE HELADO 220V",
    "MEZCLADOR PARA TALADRO TIPO HELICE 10 CM",
    "DIGESTOR DK20",
    "FRONT WALL LINERS 6008",
    'REVOLVEDOR PARA PINTURA 2X15"',
    "SISTEMA DE FABRICACIoN DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACION DE ALIMENTO BALANCEADO PARA CAMARoN A TRAVeS DE LA MOLIENDA (DESMONTADO)SISTEMA DE FABRICACIoN DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACION DE ALIMENTO BAL",
    "ICE CRUSHER",
    "AGITADOR VRP0509S120",
    "AGITADOR VRP0509S150",
    "MOLINO PLASTICO",
    "VALVULA VENTURI PARA MEXCLAR OZONO CON AGUA",
    "MAQUINA TRITURADORA DE HIELO",
    "AGITADOR VORTEX CLASSIC",
    "MEZCLADORA ESPIRAL 30 LITROS",
    "PLACA DE AGITACION PARA LABORATORIO",
    "EQUIPO HOMOGENIZADOR",
    "SHAKER",
    "AGITADOR MAGNETICO DE LABORATORIO",
    "MEZCLADOR BASICO PARA LABORATORIO",
    "ACCESORIOS AGITADOR DE TUBOS",
    "MAQUINA AGITADORA DE POLVO",
    "ROOF LINER 3298",
    "EQUIPO SIMULADOR",
    "AGITADOR MAGNETICO PARA LABORATORIO",
    "PULVERIZADORA",
    "LABORATORY MILL ZM200",
    "MEZCLADOR DE LECHE",
    "PLANCHA DE AGITACION MAGNETICA DIGITAL",
    "HOMOGENIZADOR DE MOLINO",
    "Mezclador",
    "PALETAS PARA AIREADOR",
    "MEZCLADORA UNIVERSAL 10 LITROS",
    "MEZCLADORA UNIVERSAL 20 LITROS",
    "SISTEMA DE FABRICACIoN DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACION DE ALIMENTO BALANCEADO PARA CAMARoN A TRAVeS DE LA MOLIENDA (DESMONTADO)SISTEMA DE FABRICACIoN DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACION DE ALIMENTO BAL)",
    "MULTI AGITADOR DISENADO PARA CALOR FRIO",
    "DOSIFICADOR DE CLORO",
    "VIBRATOR SYNTRON MAGNETIC FMC",
    "Sieve Shaker AS 200 control 100-240 V 50/60 Hz incl.test report acc. EN 10204 2.2",
    "MEZCLADORA UNIVERSAL 30 LITROS",
    "Agitador Mezclador Harbil 5G",
    "MAQUINA GRANIZADORA MANUAL",
    "HOMOGENIZADOR TUBOS",
    "AGITADOR DE PLACA CALEFACTORA MAGNETICA",
    "APARATO AGITADOR CON CENTRIFUGACION",
    "BLENDER",
    "MEZCLADOR DE VORTICE",
    "PICADOR DE PELLET PLASTICO",
    "MAQUINA PELADORA",
    "MOLINO DE LABORATORIO",
    "BATIDORA DE HUEVOS",
    "GRANULADOR MEZCLADOR",
    "PLATO AGITADOR CALENTADOR",
    "TRITURADORA DE MADERA",
    "TRITURADOR MANUAL",
    "SUGAR COATING MACHINE",
    "MICROPIPETA 8 CANALES VOLUMEN VARIABLE PARA LABORATORIO",
    "SISTEMA TORKMIX",
    "DISPENSADOR PLASTICO",
    "MIXER GRINDER MACHINE",
    "MAQUINA PARA MEZCLAR",
    "MEZCLADOR PARA AGUAS RESIDUALES",
    "PICADOR",
    "AGITADOR ORBITAL DE LABORATORIO",
    "EQUIPAMENTO INDUSTRIAL - SISTEMA DE PREPARACION DE ALMIDON PARA SPEED SIZER",
    "MEZCLADOR D-30 MONOFASICO 528001",
    "LAS DEMAS MAQUINAS Y APARATOS PARA MEZCLAR",
    "GRINDER BORN TO BE A GROWER",
    "3.5 HP 440V TRIFASICO 8 IMPULSOR RUEDA DE PALETA",
    "TANQUE HOMOGENIZADOR",
    "MEZCLADOR PARA CREMA LAVA VAJILLA",
    "TANQUES PARA AGITACIoN",
    "BATEDOR INDUSTRIAL",
    "AMASADORA 30LIT",
    "PRENSA HIDRAULICA MANUAL DE LABORATORIO 15 TON",
    "BATIDORA 20 LIT",
    "AGITADOR ORBITAL Y LINEAL",
    "MOTOVIBRADOR",
    "AGITADOR MECANICO DIGITAL",
    "MAQUINA AGITADORA DE ELICE",
    "INCUBADORA CON AGITACION ORBITAL DE 30 A 250 RPM",
    "MUD HOPPER JET NIPPLE P/N: 0210-112 - PI",
    "AGITADOR MECANICO",
    "MAQUINA COMERCIAL DE LECHE DE SOJA",
    "TAPAS PARA MEZCLAR",
    "CAJA 2HP 460V",
    "MEZCLADOR DE PLASTICO",
    "BATIDORA 30 LIT",
    "REACTOR DE FERMENTACION ASISTIDA",
    "BOLSAS",
    "TOLVA SECADORA",
    "EQUIPO DE LIMPIEZA DE ULTRASONIDO",
    "Knife GM 200 stainless steel",
    "MICROPULIDORA",
    "DISPENSADOR AUTOMATICO",
    "MOLINO ELECTRICO ACERO INOXIDABLE CON PARTES Y PIEZAS",
    "RETSCH Test Sieve ASTM E 11 8x2 300?m 50",
    "MINIMEZCLADOR VORTEX FISHERBRAND",
    "MAQUINA TRITURADORA DE GRANOS CON PARTES Y PIEZAS",
    "PLATO DE AGITACION",
    "ULTRASONIC PROCESSOR",
    "MAQUNA DE HIELO",
    "TANQUE DE 100L",
    "MOLINO PULVERIZADOR",
    "MEZCLADOR DE BEBIDAS",
    "VORTEX MIXER PV1",
    "MAQUINA HOMOGENEIZADOR",
    "AGITADOR CALENTADOR HSC",
    "AGITADOR ARE",
    "MEZCLADOR DE GOMA",
    "UNIDAD DE MEZCLA PARA COMBUSTIBLE",
    "AGITADOR DE PINTURAS",
    "PLOWED ICE MACHINE",
    "VIBRATOR UNIT MAQUINA CRIBADORA",
    "INCUBADORA",
    "HORNO CONTINUO",
    "TUBE ROLLER",
    "INCUBADORA CON AGITACION ORBITAL DE 30 A 250 RPM Y TEMP AMB 7 A 55 C",
    "MOLINILLO DE ALUMINIO",
    "PANEL DE CONTROL",
    "RASPADORA",
    "AGITADOR DE TECHO VERTICAL DIGITAL CON ACCESORIOS HELICE Y SOPORTE",
    "DIVISOR DE MASA DE USO INDUSTRIAL HSIAP",
    "Spiral Mixer",
    "TURBOMIX",
    "TRITURADOR DE DESPERDICIOS (INCLUYE ACCESORIOS) INSINKERATOR",
    "MEZCLADOR HORIZONTAL",
    "OCTAPOL ROTARY MIXER TUMBLER - 120V",
    "APARATO MEZCLADOR",
    "TANQUE PARA MEZCLAR",
    "MAQUINA DE HIELO 330W",
    "MEZCLADOR ESTATICO 1",
    "VORTEX FIJO 3000 RPM 110 240V",
    "MOLINO PLASTICO EUROTECNO",
    "CONJUNTO AGITADOR L592 DE SALMUERA",
    "DISPERSADOR",
    "LINEA DE MEZCLADO-SECADO-GRANULADO PARA LA PRODUCCION DE FARMACOS",
    "EQUIPO PARA CLASIFICACION DE PRODUCTOS",
    "SUBMERSIBLE MIXER 60HZ",
    "MAQUINA DE ALIMENTO PARA PECES",
    "MAQUINA PARA TRITURAR MADERA",
    "ZARANDA",
    "APARATO INDUSTRIAL PARA MEZCLAR",
    "115 VOLT MIXER FOR MUD CUP",
    "AGITADOR VRP0509S100",
    "MAQUINA DE HIELO 260W",
    "ORBITAL SHAKER",
    "APARATO FLOCULADOR DE AGITACION PARA PRUEBA DE 6 PUESTOS",
    "MAQUINA EMULSIONADORA",
    "CRIBA VIBRADORA",
    "MAQUINA TRITURADORA DE SAL",
    "MEZCLADORA VERTICAL",
    "ROTADORES DE CARRETILLA ELEVADORA",
    "MOLINO TRITURADOR PARA PLASTICO",
    "S SERIES FOOD MIXER",
    "TAMIZ ROTATIVO",
    "MAQUINA PARA TRITURAR",
    "HORNO ELECTRICO",
    "Licuadora",
    "MIXER TOPOS 1/8 BBL 200V, 50/60HZ, 3PH",
    "NIPRO HYDRATATION SYSTEM FOR DRY CONCENTRATES 75 GAL/110V",
    "MANUAL DISPENSER",
    "MAQUINA HOMOGENIZADORA",
    "DISPERSOR DE ALTA VELOCIDAD",
    "INCUBADORA CON AGITACION ORBITAL DE 30 A 250 RPM Y TEMP AMB 7 A 55C",
    "REDUCTOR CUERPO PLASTICO",
    "MAQUINA MEZCLADORA PARA PLASTICO ( MATERIAS PRIMAS )",
    "MEZCLADOR HOMOGENIZADOR",
    "TAMIZADOR EN FORMA DE ANILLO",
    "BIO REACTOR / FERMENTER",
    "MAQUINA AGITADOR DTF",
    "MAQUINA PARA TINTAS CE-MN12-102",
    "MEZCLADOR DIGITAL",
    "PARTE RASPADORA DE COCO",
    "MOLINO PICADOR",
    "SISTEMA ALMACENAMIENTO PISO MOVIL MF-Q080-12T-L",
    "MEZCLADOR PARA TANQUES",
    "ACCESORIO PARA BATIDOR",
    "CLAPETA MOTORIZADA",
    "MEZCLADOR ELETRICO PARA MORTERO HM-140 (127V)",
    "MAQUINA RASPADO DE HIELO",
    "ONILAB AGITADOR DIGITAL",
    "MEZCLADORA DIGITAL LCD CON BARRA",
    "DOUGH MIXER",
    "MEZCLADOR DE ROTACION TUBOS",
    "ROTADOR HORIZONTAL",
    "APARATO FLOCULADOR DE AGITACION PARA PRUEBA DE 4 PUESTOS CON ACCESORIOS",
    "HOMOGENIZER 250",
    "DESHUMIDIFICADOR DE AIRE PARA GENERADOR OZONO",
    "TETRA PAK HIGH SHEAR MIXER",
    "CORDL. MIXER NMX 6-22 CAJA",
    "MAQUINA MEZCLADORA DE PINTURA MANUAL",
    "LICUADORA COMERCIAL",
    "MAQUINA PULVERIZADORA",
    "ROTATING MIXER MX RD E",
    "KIT MAQUINA PARA TRITURAR",
    "AGITADOR SALMUERA TORRETA",
    "PICAHIELO",
    "BP ELECTRIC MIXER",
    "PLATO AGITADOR CALENTADOR DIGITAL",
    "LABORATORY SIEVE SHAKER AS200 CONTROL",
    "AGITADOR MAGNETICO 110V USA PLUG",
    "VIBRADOR",
    "MOLINO DE CAFÉ",
    "EQUIPO HOMOGENEIZADOR",
    "APARATO PARA MEZCLAR",
    "BOQUILLA EYECTORA CON TUBO DE GAS",
    "ANILLO ROTOR",
    "MEZCLADOR DOSIFICADOR DE AGUA",
    "MS7 HM550 PRO LCD DIGITAL PARA AGITACION Y MEZCLA",
    "CRUSHER",
    "TRITURADORA DE PLASTICO",
    "DOSIFICADOR AUTOMATICO EVO1ST",
    "MAQUINA PELADORA DE FREJOL",
    "Maquina trituradora de hielo industrial HCCP-6S",
    "Maquina trituradora de hielo industrial HCCP-6P",
    "SMALL LAB VORTEX",
    "FDS SUPER FINESS PIN MILL",
    "MAQUINA DE MOLIENDA",
    "MEZCLADORA BATIDORA",
    "RODAMIENTO DE BOLAS",
    "AGITADOR DE PLACA CALIENTE 220V",
    "AGITADOR DE PLACA CALIENTE 110V",
    "RETSCH Test Sieve ASTM E 11 8x2 75?m 200",
    "Ring sieve for ZM 200 / ZM 100 stainless",
    "TRITURADOR INDUSTRIAL DE DESPERDICIOS",
    "MEZCLADOR CON ROTOR Y ROTOR DE REPLAZO Y ACCESORIOS, USADO",
    "AGITADOR DE MICROPLACAS",
    "BOOSTER",
    "MEZCLADORES VERTICALES",
    "MAQUINA GRANULADORA",
    "Licuadora para comida de bebe",
    "Licuadora industrial",
    "MEZCLADOR ROTATORIO",
    "AGITADOR DE TINTA",
    "PULVERIZADOR",
    "INSTRUMENTOS PARA MEZCLAR ALBAÑILERIA",
    "BATIDORAS DE 7 LITROS",
    "MAQUINA MEZCLADORA HOMOGENIZADORA",
    "MIXER,BATCH TM VERT 100 BBL",
    "CONSTANT SPEED MIXER CHANDLER",
    "1. MAQUINA PARA TINTAS CE-MN10-85",
    "MAQUINA MEZCALDORA",
    "AGITADOR DE PLAQUETAS",
    "EMBARQUE PARCIAL DE SISTEMA DE FABRICACION DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACIoN DE ALIMENTO BALANCEADO PARA CAMARON (SIN MONTAR) INCLUYE EQUIPOS",
    "EQUIPO PARA LABORATORIO-MIXER",
    "Knife Mill GM 200 220-240 V 50/60 Hz",
    "equipos procesadores ultrasonidos",
    "BARRA AGITADORA MAGNETICA",
    "MEZCLADORA ONYX MIXER",
    "AGITADOR AMAMIX",
    "MAQUINA PARA TINTAS CE MN10 85",
    "MOLINO DE BOLA",
    "FLOCULADOR ESPECIAL DE AGITADOR",
    "AGITADOR DE LABORATORIO CON ACCESORIOS",
    "MANTA AGITADOR BIOBASE HMD II 2000ML DIGITAL DE CALENTAMIENTO",
    "MANTA AGITADOR BIOBASE HME II 1000ML ANALOGA DE CALENTAMIENTO",
    "MANTA BIOBASE HMD I 1000ML DIGITAL DE CALENTAMIENTO PANTALLA LED",
    "MOTOR AGITADOR BIR/6T 3SALIDAS 115V-60HZ",
    "MAQUINA PARA FABRICAR PASTA DENTAL",
    "agitador mezclador de laboratorio",
    "AGITADOR DE PLACA CALIENTE DE LABORATORIO",
    "MN10-80 MAQUINA IXELL DISTRIBUCION",
    "REDUCER TYPE H WITH PLASTIC COVER",
    "SISTEMA DE DIGESTION",
    "AGITADOR MAGNETICO PEQUENO",
    "ETP MIXER BEAR/VARI 40L WITH MK-IV AND SAFETY GRID,190-230V/50HZ/3PH AND ACCESORIES",
    "MIXER TOPOS 1/8 BBL 200V 50/60 HZ 3PH",
    "MINI ROTADOR MEZCLADOR PARA LABORATORIO",
    "AGITADOR DE HELICE COMPLETO OST-25",
    "LAS DEMAS MAQUINAS Y APARATOS",
    "MEZCLADORA DE PINTURA",
    "MAQUINA GRANIZADORA SM-102",
    "MANTA BIOBASE HME I 1000ML ANALOGICA DE CALENTAMIENTO",
    "SHAVED ICE CRUSH MACHINE",
    "SPIRAL MIXER",
    "16WHEELS WITHOUT GEARBOX AND DIESEL ENGINE WITHOUT REDUCER SHAFT CUT INTO 2PCS RE CONNECTED BY SHAFTS S304 PIPE CONNECTOR WITH FLAT RING ON THE PADS",
    "AGITADOR VIBRACIONAL",
    "PLANCHA DE AGITACION CON CALENTAMIENTO 10X10 400C 1500 RPM 115V",
    "SPIDER (ARANA)",
    "PLANCHA DE AGITACION MAGNETICA 100 1500 RPM DIGITAL 13 5 CM DIAM 115V",
    "OSCILLATOR GRANULATOR",
    "TRITURADORA DE DESECHOS DE MADERA",
    "BATIDORA DE 5 LITROS",
    "DOSIFICADOR MANUAL",
    "TOOL KIT TO REMOVE SEAL(HERRAMIENTA PARA REMOVER SELLO)",
    "REBUILD KIT FOR THE MECHANICAL SEAL(KIT DE RECOSTRUCCION DE SELLO)",
    "BATIDORAS INDUSTRIALES",
    "BEARING SET FOR THE GEARBOX(JUEGO DE RODAMIENTOS PARA CAJA DE ENGRANAJES)",
    "16WHEELS WITHOUT GEARBOX AND DIESEL ENGINE WITHOUTREDUCER SHAFT CUT INTO 2PCS RE CONNECTED BY SHAFTSS304 PIPE CONNECTOR WITH FLAT RING ON THE PADS",
    "GEAR SET(JUEGO ENGRANAJES)",
    "MECHANICAL SEAL(SELLO MECANICO)",
    "INPUT COUPLING(ACOPLE ENTRADA)",
    "AGITADOR MAGNTICO PEQUENO",
    "DISPENSADOR MANUAL",
    "MEZCLADOR PARA PLANTA DE TRATAMIENTO DE AGUA",
    "MESCLADOR DE PALETAS",
    "MOTOR AGITADOR BIR/ 7T 3SALIDAS 7M 115V",
    "PICADOR DE HIELO",
    "FLOCULADOR ESPECIE DE AGITADOR",
    "RASPADORA DE HIELLO DE 2 CUCHILLAS",
    "MAQUINA AGITADORA POLVO",
    "APARATO MEZCLADOR VORTEX",
    "TRITURADOR DE PLASTICO",
    "MODULO AGITADOR DESEQUILIBRADO",
    "PLANCHA DE AGITACION INDUCTIVA BD 600",
    "AGITADOR DE SOBREMESA PARA LABORATORIO",
    "CEBADERO",
    "EQUIPO DE PREPARACION Y DOSIFICACION DE CLORURO FERRICO",
    "MIXER (MACHINE 200 KG)",
    "TUBOS METALICOS para MONTAJE DE LANZA",
    "PICADOR DE PELLET",
    "HEATING MIXER",
    "REACTOR EMULSIFICADOR",
    "VARILLA DE AGITACION MUTZ40X15.02 02",
    "MANGUERA 10MM AZUL",
    "MANGUERA 6MM AZUL",
    "MANGUERA 8MM AZUL",
    "BATIDOR PROFESIONAL BRISK SV 50CM 350W REF: 69775",
    "AGITADOR VORTEX MX S DE 110V 60HZ USO DIDACTICO",
    "MEZCLADOR MINIMIXER TINTEC",
    "TINTEC MEZCLADOR M4 S GRIS",
    "Agitador magnetico con calefaccion Hei-PLATE Mixn Heat",
    "MANGUERA ESPIRAL 8MM AZUL",
    "OVER SIZE SIEVING MACHINE",
    "SHREDDER MACHINE",
    "PULVERIZADORA NEUMATICA TM ANDINA",
    "DESINTEGRADOR",
    "MAQUINA MEZCLADORA DE TINTA",
    "MOLINILLOS DE ALUMINIO",
    "CBZRJ-3OO VACUMM HOMOGENIZA MACHINE",
    "PLATO CALENTADOR AGITADOR MS7-H550",
    "PLANETARY BAILL MILL PM 100",
    "Electric mixer",
    "MAQUINA PARA ALISAR MUESTRAS DE LABORATORIO COMPLETA CON SUS ACCESORIOS",
    "Knife Mill GM 200 220-240 V 50 60 Hz",
    "MAQUINA MEZCLADORA DE POLIPROPILENO",
    "BEBEDERO NEPTUNO",
    "AGITADOR MAGNOTICO CON CUBIERTA ABS115 V SAT41103806",
    "OVERHEAD STIRRER. AGITADOR DE CABEZA",
    "TANQUE DE MEZCLA DE LIQUIDOS",
    "Mezclador de laboratorio",
    "MOLEDORES METALICOS, PARTES Y ACCESORIOS",
    "MAQUINA DE VASO GIRATORIO PARA PULIR JOYAS",
    "PELLET MAKING MACHINE",
    "CONTADOR DIFERENCIAL",
    "UNIT,HYDRATION TRLR 200BBL C13 T3",
    "AGITADOR GIROSCOPIO 3 VELOC",
    "AGITATOR WASHING MACHINE MODEL RF11 DIAMETER 400MM 11Z",
    "W M BOTTOM AGITATOR WASHING MACHINE MODEL B16 DIAMETER 380MM",
    "W M BOTTOM AGITATOR WASHING MACHINE MODEL A329 DIAMETER 358MM 11Z",
    "W M BOTTOM AGITATOR WASHING MACHINE MODEL A295 DIAMETER 358MM 11Z",
    "VIBRADOR DE YESO",
    "M-6610 2.4M WATERBORNE MACHINE EXP MOTOR",
    "BOLA DE GOMA",
    "1SET MAQUINA DE PRUEBAS MATERIALES PETREOS",
    "JUEGOS DE EMBUDOS",
    "Molino para laboratorio",
    "APLSTA BOTELLA",
    "SILO MEZCLA",
    "PARTE DE APARATO DE MEZCLA DE LABORATORIO DENTAL",
    "RASPADORA DE HIELO DE 4 CUCHILLAS - 110V",
    "MIXING DRUM",
    "MEZCLADOR DE LABORATORIO SILVERSON CON ACCESORIOS MODELO L5M-A",
    "DISK SCREENER",
    "TOLVA",
    "SECADOR MEZCLADOR SERIGRAFICO",
    "ACCESORIOS DE CONCENTRADOR",
    "Tapas para PerfitAll Tapa L10 para galon",
    "Tapas para PerfitAll Tapa L10 para litro",
    "Equipo mezclador PerfitAll Ancho 12 gal 7 estante",
    "MEZCLADORA ESPIRAL 40 LITROS",
    "MEZCLADOR PEQUENO",
    "DOSIFICADOR SIMULTANEO AUTOMATIC DISPENSER",
    "DOSIFICADOR AUTOMATICO SECUENCIAL AUTOMATIC SEQUENTIAL DISPENSER",
    "MOLINO ANALITICO PARA LABORATORIO",
    "EQUIPO PARA MEZCLAR",
    "TRITURADOR MARTILLO CON SUS PARTES PARA SU FUNCIONAMIENTO",
    "PLANTA COMPLETA DE TRITURADORA Y CRIBADORA DE PIEDRA CON SUS PARTES PARA SU FUNCIONAMIENTO",
    "Agitador magnetico",
    "MEZCLADOR 1400W - 800RP MINGCO",
    "FLOCULADOR JLT6",
    "FLOCULADOR FC4S",
    "AGITADOR VORTEX MIXER TX4",
    "Homogeneizador de Laboratorio con sus accesorios",
    "AGITATOR BAR",
    "MINI UNCUBATOR",
    "MOTOR AGITADOR BIR/11T 3SALIDAS 11M 115V",
    "PALETAS AIREADOR",
    "FLOTADOR 3",
    "PISTOLA DE APLICACION PARA CARTUCHO F3X8",
    "TRITURADOR DE PAVIMENTO NEUMATICO",
    "MAQUINA TRITURADORA DE TELA",
    "MAQUINA DE RECUBRIMIENTO DE AZUCAR",
    "TANQUE MEZCLADOR DE ACERO INOXIDABLE",
    "CONO TRIRURADOR",
    "LICUADORA INDUSTRIAL-BLENDER",
    "PLANCHA DE AGITACION CON CALENTAMIENTO 7x7",
    "AGITADOR CON INCUBACION, PARA LABORATORIO",
    "AGITADOR PARA TUBOS MX-SUSA,AGITADOR MAGNETICO CON CALENTAMIENTO MS7-H550-S",
    "ACCUPOINT ADVANCE SURFACE SAMPLERS 100CT Y ACCUPOINT NEXT GENERATION",
    "MEZCLADOR AQUATICLEAR 3400 PARA EL TRATAMIENTO DE AGUAS RESIDUALES",
    "PELADORA DE PAPAS DE 15 LITROS QJH-X15C",
    "PICATODO DE 8 LITROS QJH-QS8A",
    "PICATODO DE 5 LITROS QJH-QS5A",
    "TAMIZADORA ELECTRICA",
    "AGITADOR ORBITAL CON INCUBACION 230 V 60 HZ",
    "MAQUINA EXTRACTORA CONCENTRADOR DESTILADOR PARA OBTENCION DE EXTRACTOS FLUIDOS DE PLANTAS MEDICINALES",
    "MAQUINA PULIDORA PARA LIMPIAR TABLETAS EN EL PROCESO DE SU FABRICACION",
    "MAQUINA GRANULADORA PARA OBTENER GRANULADOS O POLVOS FARMACEUTICOS",
    "MAQUINA TABLETEADORA PARA FABRICAR FORMAS FARMACEUTICAS COMO TABLETAS O COMPRIMIDOS",
    "Agitador de helice para laboratorio",
    "TAMICES DE LABORATORIO",
    "CUCHILLAS",
    "EQUIPO FLOCULADOR",
    "DISCO DE CORTE",
    "MANTA AGITADOR BIOBASE HMD II 1000ML DIGITAL DE CALENTAMIENTO",
    "EXPRIMIDOR DE JUGO",
    "LICUADOR DE JUGO",
    "KIT DE HERRAMIENTAS PARA",
    "ROMPEGRUMOS",
    "HOMOGENIZADOR DE ALTA VELOCIDAD",
    "AGITADOR PARA LABORATORIO, USED",
    "MESCLADOR TIPO VORTEX, USADO",
    "MEZCLADOR KOBRAMIX 200 LITROS 220V NATURAL-CON TEMPORIZADOR",
    "DOSIFICADORES PINTURA",
    "ETP- MIXER,VARIMIXER 60 QUART ,MARK IV 208/60/3 NO HUB AND ACCESORIES",
    "AGITADOR DE MUESTRAS",
    "Electric Vegetable",
    "AGITADOR AUMATICO VIBRACIONAL",
    "AGILITADOR VORTEX MIXER",
    "MEZCLADOR ELECTRICO PARA MORTERO HM-140",
    "OLLA MEZC. DE EMULGENTE",
    "TANQUE MEZCLADOR BUFFER",
    "TANQUE MEZCLADOR PREMIX",
    "OLLA MEZC. DE HOMODAN",
    "MEZCLADOR KOBRAMIX 200 LITROS MANUAL NEGRO-CON SENSOR",
    "PLANTA DE TRATAMIENTO RESIDUOS DE MOLINO DE MARTILLOS",
    "HOMOGENIZADOR DE MUESTRAS",
    "FS650B 1 PLASTIC CRUSHER WITH 18.5KW MOTOR AND CABINET AND 12MM SCREEN",
    "FS800B 1 PLASTIC CRUSHER WITH 30KW MOTOR AND CABINET AND 12MM SCREEN",
    "MEZCLADORA DE ALIMENTOS PARA ANIMALES",
    "MOLINO TRITURADORA",
    "MAQUINA AGITADOR",
    "AGITADOR VERTICAL PARA TANQUE",
    "MAQUINA PARA HACER JABON",
    "MEZCLADOR POLVO/LIQUIDO",
    "I-AGITADOR MAGNETICO CON CALENTAMIENTO",
    "FUELLE DE CARGA TELESCOPICO",
    "MANUAL DISPENSER TS-212F",
    "MEZCLADOR NEC EN ACERO INOXIDABLE",
    "EQUIPO GRANULADOR",
    "VIBRATOR UNIT",
    "TRITURADORA DE MAIZ",
    "MEZCLADOR HIJ-V50 MI",
    "MAQUINA MEZCALDORA GRANULADOR",
    "LICUADORA PROFESIONAL",
    "ACCESORIO MOTOR PARA PICATODO - QS9E/12E MOTOR",
    "ACCESORIO MOTOR PARA PICATODO - QS9E/12E MOTOR",
    "MEZCLADORA DE 20 LITROS QJH-B20BA",
    "MEZCADORA QJH-B8",
    "MEZCLADORA DE 50 LITROS QJH-HS50S",
    "AGITADOR MAGNETICO CALEFACTOR",
    "MEZCLADORA DE 20 LITROS - S20S",
    "PELADORA DE PAPAS DE 30 LITROS QJH-X30C",
    "PELADORA DE PAPAS 10 LITROS QJH-X10C",
    "CONDUX 220 COMPACT",
    "Maquina continua de helado 600 l GNB U",
    "AGITADOR COMPLETO",
    "Incorporadora de frutas y granos en helados Super 600",
    "MESCLADOR HOMOGENIZADOR TIPO STOMACHER, USADO",
    "RECYCLING EQUIPMENT FOR HDPE",
    "HOMOGENEIZADOR 400 ML BAGMIXER 400P",
    "TUBO VACIO",
    "MEZCLADORA BASCULANTE",
    "MOLINO PARA CONDIMENTOS EN ACERO INOXIDABLE. 4.5HP. 220V/60HZ.",
    "TANQUE DE ADITIVOS 650L",
    "TANQUE DE TRATAMIENTO 4.5",
    "PROCESADOR DE ALIMENTOS MODELO QS13A",
    "LICUADORA 2200W",
    "MAQUINA BATIDORA DE HELADO DE PALETA",
    "HOMOGENEIZADOR PILOTO",
    "TRITURADOR INDUSTRIAL COMPACTO. 120-60-1. 3-4 HP",
    "APPLICATION GUN FOR F3X8 CARTRIDGE 0,35 KG",
    "Ultra Centrifugal Mill ZM 300 200-240 V 50/60 Hz",
    "AGITADOR MAGNETICO CON CALEFACCION Y PLACA CERAMICA",
    "CORTADORA",
    "MOTOR AIREADOR A DIESEL",
    "FC-307 VEGETABLE GRINDING MACHINE",
    "MILL ULTRA FINE",
    "AGITADRO DE TUBOS",
    "LAVADORA TURBO",
    "LIMPIADOR ULTRASONIDO",
    "VWR ANALOG VORTEX MIXER",
    "DOUBLE CONE MIXING MACHINE",
    "MEZCLADOR 1400W -800RP MINGCO",
    "WOOD CHIPPER",
    "MEZCLADOR DE POLVO SECO DE 110 GALONES MEZCLADOR DE GRANULOS",
    "ELCTRIC STIRRER",
    "JAR TEST TYPE FP4",
    "MAQUINA MEZCLADORA DE RESINA",
    "GRANULADOR DE ALAMBRE DE COBRE",
    "AGITADOR MAGNETICO ANALOGO CON CALENTAMIENTO",
    "AGITADOR MAGNETICO ANALOGO AHN",
    "AGITADOR MAGNETICO DE PLACA UNICA",
    "PICADORA DE RAMAS Y TRONCOS",
    "DISPERSOR",
    "ZARANDA VIBRATORIA",
    "MOTOR ROTATIVO",
    "MÁQUINA PELADORA DE FREJOL",
    "MEZCLADOR VORTEX CLASICO",
    "MEZCLADOR DE LABORATORIO AGITADOR MAGNTICO",
    "TAZA AUTOAGITADORA BOD SONDAS CAP KIT DE MEMBRANA",
    "REDUCER FOR 30SETS 12 IMPELLER PADDLEWHEEL AERATOR",
    "IMPELLER REGULAR",
    "ACCESORIO MOTOR PARA PCIATODO - QS9E/12E MOTOR",
    "MEZCLADORA DE 30 LITROS QJH-HS30S",
    "APARATO TAMIZADOR AGITADOR DE TAMICES",
    "Picadora de Hielo Portatil",
    "POWDER MACHINE",
    "AGITADOR MULTIREAX",
    "OSCILADOR-AGITADOR VIBRATORIO",
    "BATIDORA MEZCLADORA",
    "BRUSHING MACHINE FOR ARTIFICIAL TURF",
    "ARTIFICIAL TURF SAND FILLING MACHINE",
    "Recip Mezclador con agitador 65 ml",
    "Recip Mezclador con agitador 200 ml",
    "Soporte para Twister",
    "Twister 100 120V 50 60 Hz",
    "MAQUINA TRITURADOR DE PAPEL",
    "AGITADOR HOMOGENEIZADOR EN V",
    "AGITADOR DE BALANCEO",
    "CUCHILLAS PARA CORTE DE MAQUINA INDUSTRIAL",
    "ALISADOR EAGLE MOTOR LONCIN 6-5HP / G200F FS-PT36 DIAMETRO DE PALA 92CM- RANGO DE VELOCIDAD 60-100RPM",
    "BALL WHITE RUBBER FDA",
    "PLATO AGITADOR CALENTADOR DIGITAL MS-H340-S4",
    "CORONA DE BISEL HELICOIDAL",
    "JARTESTER PROGRAMABLE",
    "LINEA DE PRODUCCION DE PASTA DENTAL",
    "2 TRES 220V FASE 4 IMPULSOR RUEDA DE PALAS",
    "RUEDA DE PALAS 4 HP 440V TRIFASICA 1.2 IMPULSOR",
    "PLANTA DE MOLIENDA CON MOLINO DE MARTILLOS DE DIAMETRO 24 X 18 PULG",
    "PLANTA DE MOLIENDA CON MOLINO DE BOLAS DE DIAM. 5 X 10 PIES, TIPO DENVER",
    "MEZCLADORA CONCRET. HORMIGON REFRACTARIO",
    "ULTROMAT MT4000 Estación de preparación de polímeros",
    "MAQUINA AGITADORA DE LABORATORIO",
    "ENTRENAMIENTO REMOTO BASICO-2",
    "ENTRENAMIENTO REMOTO BASICO-1",
    "MOLINO DE BOLAS PLANETARIO MINIMILL 2",
    "MIXERS MAQUINAS MEZCLADORAS",
    "MAQUINA BATIDORA DE ARTEMIA SHAKER",
    "MODULO AGITADOR",
    "PICATODO DE 13 LITROS QJH-QS13A",
    "AGITADOR VORTEX TUBULAR",
    "DISPERSOR A PRUEBA DE EXPLOSION DISPERMAT SC537 EX",
    "AGITADOR CON BANO SECO PARA LABORATORIO",
    "PLANETARY BALL MILL PM 100 SPEED RATIO 12 120 V, 5060 HZ",
    "ROMPEHIELOS",
    "SIEVE SHAKER AS 200 CONTROL 100-240 V 50 60 HZ INCL.TEST REPORT ACC. EN 10204 2.2",
    "ULTRA CENTRIFUGAL MILL ZM 300 200-240 V 5060 HZ",
    "MEZCLADOR RX3",
    "agitador multireax",
    "AGITADOR MAGNETICO CON CALENTAMIENTO MS7-SUSA-AGITADOR MAGNETICO CON CALENTAMIENTO MS-H-S10USA-AGITADOR VORTEX TIPO MX-SUSA",
    "CONTENEDOR DE ABONO BOTE DE BASURA",
    "5LBS SIFTER-MAQUINA TRITURADORA",
    "MEZCLADOR DE GAS Y PETROLEO YHQ1-4",
    "MEZCLADOR DE GAS Y PETROLEO YHQ1-6",
    "MAQUINA MEZCLADORA DE POLVOS",
    "EQUIPO DE FRACTURAMIENTO",
    "MAQUINA DE HIELO EN ESCAMAS",
    "PIN TAPA",
    "ZARANDA VIBRATORIO 0,3 X 0,9 M",
    "ALIMENTADOR GRIZZLY VIBRATORIO DE 2 X 4 POTENCIA 5,0 HP",
    "RIDGID SINGLE PADDLE MIXER",
    "MEZCLADOR DE TINTA DE PIGMENTO",
    "TRITURADORA DE GRUMOS DLP",
    "MACHINE",
    "MAQUINA TRITURADORA DE UN EJE",
    "AGITADOR ORBITAL CON INCUBACION PARA LABORATORIO",
    "MANIJA PARA TURBO MEZCLADOR",
    "PIN ROTOR",
    "MOTOR AGITADOR BIR/7T 3SALIDAS 7m 115V-60HZ",
    "MOTOR AGITADOR BIR/11T 3SALIDAS 11m 115V-60HZ",
    "MOTOR AGITADOR BIR/18T 3SALIDAS 18m 115V-60HZ",
    "MEZCLADOR ROTATIVO-PLATAFORMA BASCULANTE",
    "RASURADOR DE HIELO",
    "KIT PREVENCION DE CORROSION",
    "Homogenizador digital para laboratorio-Molino para laboratorio",
    "ROBOT COUPE MMP 160VV",
    "Ultra Centrifugal Mill ZM 300 200",
    "Feeder kit DR 100 for ZM 300 220 240 V 60 Hz inc",
    "TANQUE MEZCLADOR DE 2000L",
    "MAQUINAS ELECTRICAS PARA PRODUCCION DE MIEL CON LAMINAS DE BASE DE PEINE",
    "TURBO MEZCLADOR",
    "BOTTOM HOMOGENIZATION Y STIRRING MACHINE WITH JACKET",
    "TANQUE 2500L",
    "MOTOR DE ARRANQUE CF1115",
    "PLANCHA DE CALENTAMIENTO",
    "MAQUINA MEZCLADORA TIPO V",
    "DIGITAL ORBITAL SHAKER",
    "VORTEX MIXER 110V",
    "MAGNETIC STIRRER 1 LITER 12V",
    "BIG MIXING LID 4L",
    "SMALL MIXING LID 1L",
    "SMALL MIXING LID 0 5L",
    "ALETAS ROTOR",
    "CAKE MIXER",
    "RUEDA DE PALETAS",
    "AGITADOR DE PLACA CALENTADORA",
    "PLANETARY BALL MILL PM 100 SPEED RATIO 1:-2 120 V 50/60 HZ",
    "PLANTA DE MOLIENDA CON MOLINO DE BOLAS DE DIAM. 5 X 10 PIES, TIPO DENVER",
    "Agitador Vibracional Automatico Shaker",
    "MUESTRA AIREADOR DE RUEDA DE PALAS",
    "EXTRACTOR",
    "BOQUILLAS",
    "RASPADORES MIXER",
    "ANILLO TAPA",
    "MEZCLADORA DE CEMENTO",
    "AUTO ORANGE JUICER",
    "MORINGA SHELLING MACHINE",
    "MOLINO GIRATORIO MINIMILL",
    "MAQUINA MESCLADORA DE YESO",
    "AMASADORA DE 20 LIBRAS",
    "CLASIFICADOR AUTOMATICO",
    "DESEMPLAZADOR ROTATIVO Y VIBRADO",
    "SH-4 Magnetic Stirrer",
    "SH-II-4C Magnetic Stirrer",
    "MEZCLADOR PARA HIERBAS DE TE Y POLVOS",
    "MIXER LABORATORIO MAGNETICO CON CALOR",
    "Tanque homogeneizador envasador en acero inoxidable con sus respectiva tapa neumatica y accesorios",
    "FLOCULADOR JLT4",
    "AGITADOR MULTI HS 6",
    "VORTEX MIXER VM-500S",
    "VORTEX MIXER VM-210",
    "Licuadora con Velocidad Variable",
    "20L STIRRER",
    "MAQUINA AGITADORA PARA MEZCLAR",
    "AGITADOR ORBITAL GRANDE CP 120V",
    "BLENDER JAR WITH BASE",
    "MARIPOSA CUCHILLA DE LICUA OSTER PN",
    "MEZCLADOR BIO CONO",
    "PLATO AGITADOR HSC",
    "HOMOGENIZER REMOTE CONTROLLER, WITH ST110/CLAMP/HT1025 TOOL HG-15D-SET-B, 230V",
    "MOLINO PARA RECICLAR PLASTICO",
    "BOTELLA DE ACERO INOXIDABLE",
    "MOLINILLO DE LECHE DE SOJA",
    "1000L HOMOGENIZER MIXING TANK",
    "MÁQUINA MEZCLADORA",
    "VORTEX VM 1 5 110V",
    "HINDA 200 MAQUINA VOLTEADORA DE ABONO CON SISTEMA DE PULVERIZADO ACCESORIO PARA EL HINDA 60HP",
    "MOLINO PARA TRITURAR PLASTICO",
    "MEZCLADOR DE TUBO DE SANGRE CAPACIDAD 15 TUBOS 25 CICLOS DE BALANCEO POR MINUTO",
    "ROTADOR CON PLACA MEZCLADORA DE VELOCIDAD VARIABLE DE 25 A 250 RPM 11X11",
    "Mezclador farmaceutico de polvo en acero inoxidable 304 con sus respectivos colectores yaccesorios",
    "SIEVE SHAKER AS 200 BASIC 120 V 60 HZ",
    "FEEDER KIT DR 100 FOR ZM 300 220-240 V 60 HZ INCL FEEDING KIT WITH CHUTE 75 40 MM",
    "MEZCLADOR DE VORTICE ANALOGICO",
    "PLANTA DE MOLIENDA CON MOLINO DE BOLAS DE DIAM. 5 X 10 PIES, TIPO DENVER",
    "HEI-TORQUE EXPERT 100 OH AGITADOR",
    "MEZCLADOR EMULSIONADOR AL VACIO",
    "CRIBA MOGENSEN",
    "AGITADOR500LTANK",
    "AGITADOR300LTANK",
    "AGITADOR100LTANK",
    "UNIDAD MEZCLADORA",
    "MEZCLADOR DE DISPERSION",
    "MAQUINA BATIDORA DE HELADOS",
    "MAQUINA DE ALGODON DE AZUCAR ELECTRICA 110V",
    "1 SACO 1 FUNDA DE CARAMELOS 2 PANTALONES 1 BUSO 1 BLUSA 1 FRASCO DE ASPIRINA 3 VITAMINAS 1 CASACA DE NINO",
    "6 PARES DE ZAPATOS 2 MEDIAS",
    "3 VITAMINAS, 3 PANTALONES, 1 BLUSA, 2 SPLASH, 1 CREMA",
    "3 ABRIGO 4 TOP 3 TERNO 4 BUZOS",
    "ZARANDA VIBRATORIA 3X6",
    "Knife Mill GRINDOMIX GM 300 220-230 V 50/60 Hz incl. 5 l plastic container with cover, knife of stainless steel and scraper",
    "VORTEX-3",
    "MIXER 15 MIN 1 3 HP 110 60",
    "EMBARQUE PARCIAL DE SISTEMA DE FABRICACION DE ACONDICIONAMIENTO DE MATERIA PRIMA PARA LA FABRICACIoN DE ALIMENTO BALANCEADO PARA CAMARON SIN MONTAR INCLUYE EQUIPOS",
    "SISTEMA DE DOSIFICACION GRAVIMETRICA",
    "INCUBADORA CLIMATICA DE TEMPERATURA CONSTANTE",
    "Homogenizador ultrasonico",
    "PICA TODO MODELO QS12E 220V 60HZ",
    "PICA TODO MODELOS QS9E 220V 60HZ",
    "Licuadora industrial CB-699D",
    "MOLINO DE MARTILLO",
    "ROTADOR ORBITAL AJUSTABLE VELOCIDAD 100-500RPM",
    "PAQUETE DIGITAL RW 20 S01",
    "DOSIFICADOR SIMULTANEO AUTOMATICO",
    "DOSIFICADOR MANUAL M3S 12",
    "DOSIFICADOR MANUAL MIXER",
    "DOSIFICADOR AUTOMATICO",
    "MAQUINA AGITADOR DE MUESTRAS DE SUELOS",
    "AMOLADORA PARA CACAO",
    "HOMOGENIZADOR DE LABORATORIO UXI",
    "03-COMEDEROS PARA MASCOTAS",
    "AMOLADORA ELECTRICA",
    "APARATO MECANICO",
    "1 MEZCLADOR AGITADOR DE LABORATORIO",
    "RASPADORA DE HIELO USADA",
    "MAQUINA DE TRITURAR HIELO",
    "MAQUINA DE HIELO",
    "MOLINO DE GRANOS",
    "2 BUSOS 2 CAMISETAS 3 PARES DE TENNIS",
    "MAQUINA TRITURADORA DE RAFIA",
    "MAQUINA TINTOMETRICA",
    "STIRRER ANALOG, OVERHEAD, HIGH-SPEED, 5000MPAS HS-50A-SET, W/KA1177, 230V",
    "PLANTA DE MOLIENDA CON MOLINO DE BOLAS DE DIAM. 5 X 10 PIES, TIPO DENVER",
    "1 PULSADOR, 3 ACCAESORIOS, 1 TENSOMETRO, 1 JUGUETE",
    "3 CAMISETAS 3 BUSOS 1 SET DE MEDIAS 2 BLUSAS",
    "1 CHOMPA 5 PELUCHES 5 BLUSAS 4 SHORES 2 VESTIDOS 1 CALENTADOR",
    "CAMISA,ZAPATOS,TRAJES,CHUMPAS,TRAJES",
    "PILLAMAS , PANTIS, MAQUILLAJE",
    "1 PAQUETE DE JABON 1 EDREDON",
    "3 LAMPARAS DE PERRO 5 CAMISETAS DE HOMBRES 5 BUSOS DE HOMBRES 1 PACA DE MANTELES DE PERRO",
    "PLASTIC BOTTLE SHREDDER MACHINE AND ALL ITS PARTS",
    "PULSE 65HS EU A",
    "AMASADORA DM-33 110/60/1",
    "Maquina Vibradora",
    "Agitador magnEtico con placa calefactora",
    "MAQUINAS AGITADORAS Y SUS PARTES",
    "Sample Divider PT 100 complete unit 220-240 V, 60 HZ",
    "Sieve Shaker AS 200 control 100-240 V, 50/60 Hz",
    "Ultra Centrifugal Mill ZM 300 200-240 V, 50/60 Hz",
    "RASPADORA DE HIELO MODELO T-6",
    "AGITADOR SMG",
    "MAQUINA PARA FABRICACION DE TINTAS",
    "MEZCLADOR/AGITADOR BLIXER VERTIAL",
    "APARATO AGITADOR MAGNETICO COMPLETO",
    "HOMOGENIZADORES",
    "MAQUINA DE PINTURA AUTOMATRIZ",
    "HOMEGENIZADOR- CONVERTIDOR",
    "2 UNIDADES , MAQUINA MEZCLADORA",
    "BATIDORA DE MANO",
    "AGITADOR MAGNETICO 110V",
    "200L TANQUE MEZCLADOR",
    "NEW LAB BASKET MILL",
    "DISPENSADOR COMBINADO 2 EN 1",
    "MX-S Vortex Mixer (with all adaptors)",
    "SISTEMA DE CLASIFICACION AUTOMATICA",
    "PULVERIZADOR DE RESTOS DE ANIMALES DOMESTICOS",
    "MEZCLADOR DE COLA HIDRAULICO DE ALTA VELOCIDAD",
    "TANQUE DE PRESION CON AGITADOR",
    "MAQUINA DE RECUBRIMIENTO",
    "DISPOSITIVO DE DOSIFICACION HEGA NAT",
    "AGITADOR VERTICAL VPP3 460V 60 HZ 0503B02/086S",
    "BATIDORA ELECTRICA DE MANO DE ACERO INOXIDABLE 6PZ",
    "BALL MILL BKBM-V2 (110V)",
    "CABINA DE BIOSEGURIDAD",
    "HOMEGENIZADOR",
    "CRIBA VIBRATORIA",
    "Espumador de leche",
    "MAQUINA TRITURADORA DE PLASTICO",
    "MEZCLADOR DE PINTURAS",
    "PLATO CALEFACTOR",
    "PLANTA DE MOLIENDA CON MOLINO DE BOLAS DE DIAM. 5 X 10 PIES, TIPO DENVER",
    "ASTM-COMPLIANT MIXER 5-QT. 4.73L",
    "AGITADOR TYPHOON",
    "MAQUINA DE PINTURA AUTOMOTRIZ",
    "TRITURADORA DE HIELO ELECTRICA",
    "SOAP EXTRUDER MACHINE",
    "MAQUINA PEQUENA QUE AGITA EL POLVO DE SUBLIMACION",
    "AGITADOR BASICO PARA LABORATORIO",
    "MEZCLADORA PARA INDUSTRIA ALIMENTICIA",
    "MAQUINA COCTELERA GIG-8 PRO c/8 adaptadors MAS Tablet",
    "AIREADOR PAD 4 HP SS MOTOR 12 IMP 440V",
    "AGITADOR ORBITAL ITAL",
    "ORBITALES DIGITAL",
    "CUBIERTA DIVIDIDA PARA TRABAJO PESADO PARA 1000",
    "ENGANCHE RAPIDO HEMBRA PARA DOBLE",
    "CONTENEDOR DE DISPERSION DE DOBLE PARED",
    "SISTEMA DE FRESADO TORUSMILL",
    "EQUIPO DE FIJACION PT 100",
    "TERMOMETRO DE RESISTENCIA PT 100",
    "DISPERSO DE DISOLVOR DE TRABAJO",
    "BREAKING JAWS FOR BB500",
    "V BELT BB500",
    "NEUMATIC MIXING TANK",
    "DIBUJOS DEL SISTEMA",
    "SOPORTE UNIVERSAL",
    "ADAPTADOR DE SONOTRODO ROSCADO",
    "SONOTRODO LARGO",
    "CASCATRODE",
    "PROCESADOR ULTRASONICO INDUSTRIAL CON ACCESORIOS",
    "MEZCLADOR DE ELEVACION DE ALTO CIZALLAMIENTO",
    "DOUGH SHEETER",
    "AGITADOR VORTICE",
    "AGITADOR DE HELICE COMPLETO",
    "JAW CRUSHER BB500",
    "AGITADOR ORBITAL PARA LABORATORIO, USADO",
    "MAQUINA TRITURADORA Y GRANULADORA",
    "MAQUINA MOLINILLO DE CAFE",
    "AGITATOR, LOW PROFILE, 15HP, PB123",
    "ABREBOLSAS",
    "INYECTOR VENTURI NEGRO PARA OZONO 2",
    "INYECTOR VENTURI NEGRO PARA OZONO 1 4",
    "INYECTOR VENTURI NEGRO PARA OZONO 1",
    "INYECTOR VENTURI BLANCO PARA OZONO 1 5",
    "INYECTOR VENTURI BLANCO PARA OZONO 1",
    "RECARGO DE MATERIAL",
    "MEZCLADOR ESTATICO PARA OZONO 2",
    "MEZCLADOR ESTATICO PARA OZONO 1 5",
    "MEZCLADOR ESTATICO PARA OZONO 1",
    "REPUESTO ADAPTADOR RECTO DE BRONCE PARA CONEXION DE MANGUERA",
    "BALL MILL BKBM-V2 (110V",
    "PLANTA DE MOLIENDA CON MOLINO DE BOLAS DE DIAM. 5 X 10 PIES, TIPO DENVER",
    "SEPARADOR DE TIERRA DE PAPA POST EN ACERO INOXIDABLE",
    "MATRAZ DE EVAPORACION",
    "MEZCLA DE PLATOS HEI 20L 115V",
    "LICUADORA DE ALTA POTENCIA PARA USO EN RESTAURANTE",
    "INYECTOR VENTURII NEGRO PARA OZONO AGRICOLA 1",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 1710-S02 FC 1757KG 17.2KN 1.2KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 1410-S02 FC 1449KG 14.2KN 1.05KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 900-S02 FC 1037KG 10.2KN 0.67KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 550-S02 FC 592KG 5.81KN 0.35KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 200-S02 FC 214KG 2.10KN 0.17KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 100-S02 FC 117KG 1.10KN 0.095KW",
    "AGITADOR TIPO VORTEX, DIGITAL",
    "AGITADOR DE LABORATORIO TIPO VORTEX",
    "HOTPLATE MAGNETIC STIRRER 3 MS7-H550-PRO",
    "MILK SHAKE MACHINE LICUADORA",
    "SISTEMA DE FABRICACION DE ACONDICIONAMIENTO",
    "APARATO PARA TAMIZAR",
    "SONOTRODO DE TITANIO PARA HOMOGENEIZACION POR ULTRASONIDO",
    "MESCLADOR",
    "PARTE DE APARATO PARA MEZCLAR",
    "AGITADOR ORBITAL 110 V ENCHUFE DE EE UU",
    "AGITADOR MAGNETICO 110V ENCHUFE EE UU",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 3000-S08 FC 3107KG 30.5KN 2KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 2410-S08 FC 2440KG 24KN 1.7KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 2000-S02 FC 2033KG 19.9K 1.45KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 80-S02 FC 76.1KG 0.75KN 0.095KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 35-S02 FC 43.5KG 0.43KN 0.095KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 10 3000-S02 FC 3124KG 30.6KN 2.4KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 10 550-S02 FC 737KG 7.23KN 0.38KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 3 1500-S08 FC 1595KG 15.6KN 1.3KW",
    "REPUESTOS",
    "SCREENING MACHINE",
    "SET PLANTA COMPLETA DE TRITURADORA Y CRIBADORA DE PIEDRAS CON SUS PARTES",
    "PLATO CALENTADOR AGITADOR MS7 H550",
    "AGITADOR VORTEX TIPO MX SUSA",
    "HOMOGENIZADOR D 160",
    "AGITADOR MAGNETICO CON CALENTAMIENTO MS H S10USA",
    "MEZCLADORES DE PLASTICO",
    "LAMINADORA DE FILM PLASTICO",
    "ROLL SCREEN",
    "AGITADOR TIPO VORTEX, USADO, RUO",
    "IMAN ROTATIVO DFRT-500/1220 PL",
    "TAMIZ ROTATIVO LAKA 400",
    "MEZCLADOR DIGITAL PARA LABORATORIO",
    "AGITADOR DE TUBOS CON CONTROL DE TEMPERATURA",
    "PLANTA DE MOLIENDA CON MOLINO DE BOLAS DE DIAM. 5 X 10 PIES, TIPO DENVER",
    "HOMOGENIZER",
    "HOMOGENIZAR D-160",
    "AGITADOR VORTEX TIPO MX-SUSA",
    "AGITADOR DE TUBOS MX-RL-PRO",
    "AGITADOR ORBITAL, USADO, RUO",
    "DISPOSITIVO GIRATORIO COMO AYUDA PARA EL CAMBIO DEL TAMIZ ROTATIVO",
    "PRM-2 RIBBON MIXER",
    "MAQUINAS MEZCLADORAS DE AMASADO O AGITACION",
    "CORTADOR BATIDORA CUTTER BLENDER FRUITS",
    "MEZCLADOR JQ500",
    "PADDLEWHEEL AERATOR COMPONENT(CONNECTION POST)",
    "PADDLEWHEEL AERATOR COMPONENT(CONNECTION BOX)",
    "PADDLEWHEEL AERATOR COMPONENT(FLOAT)",
    "GSC-2.2 440V 60HZ 3PHASE",
    "MOTOVIB 3F 230 460V 60HZ MVSI 3 200-S02 FC 234KG 2.14KN 0.18KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 3 100-S02 FC 176KG 1.71KN 0.18KW",
    "APARATO ELECTRICO PARA MEZCLAR",
    "729910000 MIXER - AMAMIX C 2227/24 UDG",
    "729696000 MIXER - AMAMIX C 2222/14 UDG",
    "PULVERIZADORA NEUMATICA",
    "CABEZAL DE REPUESTO PARA L VM1000",
    "MEZCLADORA DE TUBOS DE SANGRE CAPACIDAD 15 TUBOS VIALES",
    "MOTOVIB 3F 230 460V 60HZ MVSI 3 300-S02 FC 326KG 2.91KN 0.27KW",
    "LICUADORAS PARA USO INDUSTRIAL",
    "MIXER 15 MIN 1 3HP 110 60 W GUARDS",
    "MAQUINA ZARANDA PARA COMPOSTERA",
    "INCUBADORA DE TEMPERATURA CONSTANTE",
    "INCUBADORA CON AGITACION ORBITAL DE ROSCA BSPT DEL SISTEMA DE FILTRACION TE 0593",
    "Maquina HA480 32 depositosina tintometrica",
    "AGITADOR ELECTRICO",
    "MARTILLOS DE MOLINOS PARA MEZCLAR",
    "EMULSIONADOR TRITURADOR",
    "HOMOGENIZADOR TIPO Vo DE CALZON",
    "BATIDORA DE HELADO CON PRECCOLING",
    "BATIDORA DE HELADO GMCC",
    "LICUADORA 2,2KW",
    "MAQUINAS DE RASPADO DE HIELO CON COCHE",
    "HUMAX 3K",
    "COATING MACHINE",
    "AGITADOR ENCHAQUETADO CREMA",
    "AGITADOR DE MESA PARA COSMETICOS Y CREMAS",
    "AGITADOR DE MESA PARA CREMA",
    "MOTOVIB 3F 230 460V 60HZ MVSI 3 500-S02 FC 504KG 4.62KN 0.5KW",
    "MEZCLADOR DE HARINAS CAPACIDAD 100 KG",
    "MEZCLADORES DE AGUA",
    "AGITADOR DE LABORATORIO PARA MUESTRAS DE AGUA",
    "AGITADOR VORTEX 110V",
    "PLANTA DE MOLIENDA CON MOLINO DE BOLAS DE DIAM. 5 X 10 PIES, TIPO DENVER",
    "MOTOVIB 3F 230 460V 60HZ MVSI 15 2000-S02 FC 2033KG 19.9K 1.45KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 3 1100-S02 FC 1061KG 10.4KN 1.2KW",
    "MOTOVIB 3F 230 460V 60HZ MVSI 3 800-S02 FC 754KG 7.40KN 0.68KW",
    "AGITADOR DE TECHO VERTICAL DIGITAL CON ACCESORIO SOPORTE Y PLATO",
    "LICUADORA ELECTRICA INDUSTRIAL",
    "MOLINO DE HIELO PARA MAQUINA PARA HACER HIELO",
    "MEZCLADOR DE BOLSAS",
    "MINI AGITADORA",
    "MAQUINA AGITADORA DE PINTURA",
    "HOMOGENIZADOR PROCESADOR DE ALIMENTOS PARA RESTAURANT",
    "AGITADOR DE TAMICES",
    "AGITADORES ORBITALES 13X12",
    "TRIPLE GROUSER BAR",
    "TOLVA DE BIOMASA",
    "AGITADOR OSCILATORIO DIGITAL",
]
# Filter out unwanted values from the 'DESCRIPCION PRODUCTO COMERCIAL' column
df = df[~df["DESCRIPCION PRODUCTO COMERCIAL"].isin(filter_out)]
# Map the 'DESCRIPCION PRODUCTO COMERCIAL' column using the description_mapping dictionary
df["DESCRIPCION PRODUCTO COMERCIAL"] = df[
    "DESCRIPCION PRODUCTO COMERCIAL"
].replace(description_mapping)
# Display the most common values in the 'DESCRIPCION PRODUCTO COMERCIAL' column again after mapping
print("Most common values in 'DESCRIPCION PRODUCTO COMERCIAL':")
value_counts = df["DESCRIPCION PRODUCTO COMERCIAL"].value_counts().head(50)
for value in value_counts.index:
    print(value)
# Print the shape of the DataFrame after filtering and mapping
print(f"Shape of DataFrame after filtering and mapping: {df.shape}")


Most common values in 'DESCRIPCION PRODUCTO COMERCIAL':
AIREADOR
FLOCULADOR 6 PUESTO HORIZONTAL
FLOCULADOR 4 PUESTO HORIZONTAL
AGITADOR VORTEX COMPLETO
TAMIZADORA CIRCULAR
AGITADOR DE CAFÉ
AGITADOR MECANICO VERTICAL DE LABORATORIO
MAQ. MEZCLADORA REACTOR CON SUS PARTES Y PIEZAS PARA INSTALACION Y FUNCIONAMIENTO.
AGITADORES MAGNETICOS CON CALENTAMIENTO INCLUYE SONDA DE TEMPERATURA
MAQUINA CORTADORA D/PIPAS
VWR SHAKER 3500 STANDARD 120V
AGITADOR DE TAMIZ DE PRUEBA TAMIZADOR ROTAP
APARATO DIGITADOR DE TAMIZ GRUESO
HOMOGENIZADOR DE ALTO CIZALLAMIENTO DE TRES ETAPAS
MAQUINA PELETIZADORA DE PLASTICO
MOLINO DE BOLAS RODANTE PARA CERAMICA
REVESTIMIENTO DE MOLINO DE BOLAS
MOLINO DE BOLAS CON ARRANCADOR
AGITADOR DE JARRAS DE LABORATORIO
MOLINO PARA ESPECIES
PALETA DE RUEDA
16WHEELS WITHOUT GEARBOX AND DIESEL ENGINE WITHOUTREDUCER SHAFT CUT INTO 2PCS RE CONNECTED BY SHAFT SS304
MEZCLADORA VORTEX
MEZCLADOR PORTATIL PARA LIQUIDOS ESPESOS
MEZCLADOR PORTATIL PARA LIQUIDOS LIVIANOS
GRINDING MACHINE
TA

In [167]:
# Keep only the rows where 'DESCRIPCION PRODUCTO COMERCIAL' value is "AIREADOR"
df = df[df["DESCRIPCION PRODUCTO COMERCIAL"] == "AIREADOR"]
# Display the shape of the DataFrame after filtering for "AIREADOR"
print(f"Shape of DataFrame after filtering for 'AIREADOR': {df.shape}")
# Display the first few rows of the filtered DataFrame
print("Filtered DataFrame for 'AIREADOR':")
print(df.head())
# Display the most common values in the 'DESCRIPCION PRODUCTO COMERCIAL' column
print("Most common values in 'DESCRIPCION PRODUCTO COMERCIAL' after filtering for 'AIREADOR':")
value_counts = df["DESCRIPCION PRODUCTO COMERCIAL"].value_counts().head(50)
for value in value_counts.index:
    print(value)


Shape of DataFrame after filtering for 'AIREADOR': (650, 57)
Filtered DataFrame for 'AIREADOR':
    DIA  MES   AÑO       REGIMEN ADUANERO              REFRENDO     DAU  ITEM  \
2    27    1  2021  IMPORTACION A CONSUMO  028-2021-10-00070000   287.0     1   
3     7    1  2021  IMPORTACION A CONSUMO  028-2021-10-00015273   939.0     1   
4     7    1  2021  IMPORTACION A CONSUMO  028-2021-10-00014969  3482.0     1   
35   26    1  2021  IMPORTACION A CONSUMO  028-2021-10-00065351   144.0     1   
41    7    1  2021  IMPORTACION A CONSUMO  028-2021-10-00013812    21.0     1   

    RUC IMPORTADOR        PROBABLE IMPORTADOR PAIS DE PROCEDENCIA  \
2     992758791001               KARASTE S.A.               CHINA   
3     993250236001             ALFAGUYCA S.A.               CHINA   
4     992758791001               KARASTE S.A.               CHINA   
35    911990703001    MACIAS PIZA MARIO CESAR               CHINA   
41    992717769001  IMPORTADORA ECUASINO S.A.               CHINA   

  

In [168]:
columns_to_drop = [
    "DAU",
    "CIUDAD EMBARQUE",
    "MARCA",
    "EMBARCADOR",
    "EMPRESA DE TRANSPORTE",
    "NAVE",
    "NUMERO DE MANIFIESTO",
    "CONOCIMIENTO EMBARQUE",
    "DEPOSITO COMERCIAL",
    "TIPO AFORO",
    "DIRECCIóN CONSIGNATARIO",
    "FECHA DE EMBARQUE",
    "FECHA DE LLEGADA",
    "FECHA PAGO",
    "FACTURA",
    "CARACTERISTICAS",
    "PRODUCTO",
    "MARCA COMERCIAL",
    "AñO FABRICACIóN",
    "MODELO MERCADERIA",
    "FECHA DE LIQUIDACIóN",
    "FECHA AFORO",
    "INCOTERM",
    "NUMERO DE CARGA FINAL",
    "PAíS DE ORIGEN",
    "DESCRIPCIóN ARANCELARIA",
    "PAíS DE EMBARQUE",
    "FECHA SALIDA",
    "CóDIGO TNAN",
    "CONTENEDOR",
    "UNIDAD DE MEDIDA",
    "PARTIDA ARANCELARIA",
    "DESCRIPCION PRODUCTO COMERCIAL",
    "DIA",
    "MES",
    "AÑO",
    "REGIMEN ADUANERO",
    "REFRENDO",
    "ITEM",
    "AGENTE DE ADUANA",
    "ADUANA",
]
# Drop any remaining NaN values in the remaining columns
df = df.dropna(
    subset=[col for col in df.columns if col not in columns_to_drop]
)
# Drop columns that are not needed for clustering
df = df.drop(columns=columns_to_drop)


# Clean and convert FECHA INGRESO SISTEMA to datetime
def clean_date_string(date_str):
    """Clean date string and convert to proper format"""
    if pd.isna(date_str):
        return None

    # Convert to string and remove any non-digit characters except spaces
    date_str = str(date_str).strip()

    # If already in YYYY-MM-DD format, return as is
    if len(date_str) == 10 and date_str[4] == "-" and date_str[7] == "-":
        return date_str

    # Remove any non-digit characters
    digits_only = "".join(filter(str.isdigit, date_str))

    # Check if we have at least 8 digits for YYYYMMDD format
    if len(digits_only) >= 8:
        year = digits_only[:4]
        month = digits_only[4:6]
        day = digits_only[6:8]

        # Validate year, month, day ranges
        try:
            if int(year) < 2020 or int(year) > 2025:
                return None
            if int(month) < 1 or int(month) > 12:
                return None
            if int(day) < 1 or int(day) > 31:
                return None

            return f"{year}-{month}-{day}"
        except ValueError:
            return None

    return None


# Apply cleaning function
df["FECHA_TEMP"] = df["FECHA INGRESO SISTEMA"].apply(clean_date_string)

# Convert to datetime with error handling
df["FECHA INGRESO SISTEMA"] = pd.to_datetime(df["FECHA_TEMP"], errors="coerce")

# Drop the temporary column
df = df.drop(columns=["FECHA_TEMP"])

# Show how many dates were successfully converted
valid_dates = df["FECHA INGRESO SISTEMA"].notna().sum()
total_rows = len(df)
print(
    f"Successfully converted {valid_dates} out of {total_rows} dates ({valid_dates / total_rows * 100:.1f}%)"
)

# Print the remaining columns and missing values
print("Missing values in remaining columns:")
print(df.isnull().sum())
# Print the shape of the DataFrame after dropping columns
print("Shape of DataFrame after dropping columns:", df.shape)
# Preview the cleaned DataFrame with no truncated columns
pd.set_option("display.max_columns", None)
print("Preview of cleaned DataFrame:")
print(df.head(20))


Successfully converted 649 out of 649 dates (100.0%)
Missing values in remaining columns:
RUC IMPORTADOR           0
PROBABLE IMPORTADOR      0
PAIS DE PROCEDENCIA      0
VIA DE TRANSPORTE        0
ESTADO DE MERCANCIA      0
BULTOS                   0
CANTIDAD                 0
ADVALOREM                0
US$ FOB                  0
US$ FLETE                0
US$ SEGURO               0
US$ CIF                  0
PESO NETO KG             0
FECHA INGRESO SISTEMA    0
US$ FOB UNIT             0
AGENCIA DE CARGA         0
dtype: int64
Shape of DataFrame after dropping columns: (649, 16)
Preview of cleaned DataFrame:
     RUC IMPORTADOR                      PROBABLE IMPORTADOR  \
2      992758791001                             KARASTE S.A.   
3      993250236001                           ALFAGUYCA S.A.   
4      992758791001                             KARASTE S.A.   
35     911990703001                  MACIAS PIZA MARIO CESAR   
41     992717769001                IMPORTADORA ECUASINO S.A.  

In [169]:
# Get value counts for all columns
def get_value_counts(df):
    """Get value counts for all columns in the DataFrame."""
    value_counts = {}
    for column in df.columns:
        value_counts[column] = df[column].value_counts().head(10).to_dict()
    return value_counts

# Get value counts for all columns
value_counts = get_value_counts(df)
# Display value counts for each column
for column, counts in value_counts.items():
    print(f"Value counts for '{column}':")
    for value, count in counts.items():
        print(f"  {value}: {count}")
    print()  # Add a newline for better readability


Value counts for 'RUC IMPORTADOR':
  992717769001: 202
  992758791001: 88
  993069337001: 67
  992882638001: 41
  990018685001: 37
  992597143001: 31
  990792658001: 25
  993238538001: 19
  993064998001: 16
  791841758001: 11

Value counts for 'PROBABLE IMPORTADOR':
  IMPORTADORA ECUASINO S.A.: 198
  KARASTE S.A.: 88
  TECNIFICACIONES ACUICOLAS PROQUALIA S A: 67
  PRODUCTOS Y SERVICIOS INTEGRALES TRESVICOR S.A.: 41
  CONAUTO C.A.: 37
  COSUMISA S.A.: 31
  IMPORTADORA LINO GAMBOA CIA. LTDA. ILGA: 25
  SOLUINMAQ SOLUCIONES, INSUMOS Y MAQUINARIAS S.A.: 19
  CONSULTCOMMERCE S.A.: 16
  IMPORTADORA PROQUALIA IMPROQUALIA S.A.: 11

Value counts for 'PAIS DE PROCEDENCIA':
  CHINA: 601
  INDIA: 19
  BRAZIL: 10
  HONG KONG: 8
  UNITED STATES: 5
  VIETNAM: 2
  ISRAEL: 2
  TAIWAN: 1
  GERMANY: 1

Value counts for 'VIA DE TRANSPORTE':
  MARITIMO: 644
  AEREA: 5

Value counts for 'ESTADO DE MERCANCIA':
  NUEVA: 648
  DESARMADA/NUEVA: 1

Value counts for 'BULTOS':
  0.0: 44
  2340.0: 11
  1593.0: 8
  